In [ ]:
##Run this two times for '../Data/Predictions/Nucleus_Prediction' and '../Data/Predictions/Mitochondria_Prediction'

import os, re, gc
import numpy as np
import sys
import PathConfig
from read_write_mrc import *
dirPath = '../Data/Predictions/Nucleus_Prediction'
fileList = sorted(os.listdir(dirPath))
cellIds = {}

In [ ]:
for filename in fileList:
    cell_id = None 
    match = re.search(r'(\d+_\d+)_merged', filename)
    if match:
        cell_id = match.group(1)
    elif (match := re.search(r'(\d+_\d+)_pre_rec', filename)):  
        cell_id = match.group(1)
    elif (match := re.search(r'(\d+_\d+-\d+)_pre_rec', filename)):  
        cell_id = match.group(1)
        
    axis = re.search(r'(\w+)_patch', filename).group(1)[-1]
    patchNum = int(re.search(r'patch(\d+)', filename).group(1))
    if cell_id not in cellIds:
        cellIds[cell_id] = {}
    if axis not in cellIds[cell_id]:
        cellIds[cell_id][axis] = {}
    if "mergedfileName" not in cellIds[cell_id]:
        if "pre_rec" in filename:
            cellIds[cell_id]["mergedfileName"] = filename.split("_pre_rec")[0]+"_merged_prediction.mrc"
        else:
            cellIds[cell_id]["mergedfileName"] = filename.split("_merged_prediction")[0]+"_merged_prediction.mrc"
    if patchNum not in cellIds[cell_id][axis]:
        cellIds[cell_id][axis][patchNum] = os.path.abspath(os.path.join(dirPath, filename))
print(cellIds.keys())
len(cellIds)

In [9]:
mergeDirPath = '../Data/Predictions/PredictionMerged_Nucleus' ## Change to Mitochondria for Mitochondria_Prediction.
if not os.path.isdir(mergeDirPath):
    os.makedirs(mergeDirPath)

In [ ]:
for cellId in cellIds:
    addImage = np.zeros((704,704,704), dtype=np.float32)
    counterImage = np.zeros((704,704,704), dtype=np.float32)
    counterSlice = np.ones((64, 704, 704), dtype=np.float32)
    axis = "x"
    for patchNum in sorted(cellIds[cell_id][axis].keys()):
        filename = cellIds[cellId][axis][patchNum]
        v = read_mrc(filename)
        startIndex = patchNum*32
        endIndex = startIndex + 64
        addImage[startIndex:endIndex, :, :] += v
        counterImage[startIndex:endIndex, :, :] += counterSlice
    addImage = np.rot90(addImage, axes=(0,2))
    counterImage = np.rot90(counterImage, axes=(0,2))
    addImage = np.flip(addImage, axis = 0)
    counterImage = np.flip(counterImage, axis = 0)
    axis = "y"
    for patchNum in sorted(cellIds[cell_id][axis].keys()):
        filename = cellIds[cellId][axis][patchNum]
        v = read_mrc(filename)
        startIndex = patchNum*32
        endIndex = startIndex + 64
        addImage[startIndex:endIndex, :, :] += v
        counterImage[startIndex:endIndex, :, :] += counterSlice
    addImage = np.rot90(addImage, axes=(1,0))
    counterImage = np.rot90(counterImage, axes=(1,0))
    addImage = np.flip(addImage, axis = 1)
    counterImage = np.flip(counterImage, axis = 1)    
    axis = "z"
    for patchNum in sorted(cellIds[cell_id][axis].keys()):
        filename = cellIds[cellId][axis][patchNum]
        v = read_mrc(filename)
        startIndex = patchNum*32
        endIndex = startIndex + 64
        addImage[startIndex:endIndex, :, :] += v
        counterImage[startIndex:endIndex, :, :] += counterSlice
    addImage = addImage/counterImage
    # addImage = addImage>0.6
    filename = os.path.join(mergeDirPath, cellIds[cellId]["mergedfileName"])
    write_mrc(filename, addImage)
    print("Saved:", filename)
    del addImage, counterImage, counterSlice, v, filename
    gc.collect()